In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout
np.set_printoptions(precision=3, suppress=True)

from collect_data import vahid_data, amanda_data, manuel_data, names

from IPython.display import clear_output
clear_output()

In [2]:
input_dim = 3

encoders = []
decoders = []
autoencoders = []

input_layer = Input(shape=(input_dim,))

In [3]:
# initializer = tf.keras.initializers.GlorotNormal()
initializer = tf.keras.initializers.RandomNormal()
for i in range(216):
    encoder = Dense(64, activation='silu', kernel_initializer=initializer)(input_layer)
    encoder = Dropout(0.2)(encoder)
    encoder = Dense(1, activation='softplus', kernel_initializer=initializer)(encoder)  # Bottleneck layer (1D representation)
    encoders.append(encoder)

    # Define the decoder
    decoder = Dense(64, activation='silu', kernel_initializer=initializer)(encoder)
    decoder = Dropout(0.2)(decoder)
    decoder = Dense(input_dim, activation='linear', kernel_initializer=initializer)(decoder)
    decoders.append(decoder)

    # Define the autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=decoder)
    autoencoders.append(autoencoder)

    # Compile the model
    autoencoders[i].compile(optimizer='adam', loss='mean_squared_error')
# clear_output()

In [4]:
for i in range(6):
    for j in range(12):
       autoencoders[12*i+j].fit(vahid_data[i][j], vahid_data[i][j], epochs=100, verbose=0)
       autoencoders[12*6+12*i+j].fit(amanda_data[i][j], amanda_data[i][j], epochs=100, verbose=0)
       autoencoders[12*6*2+12*i+j].fit(manuel_data[i][j], manuel_data[i][j], epochs=100, verbose=0)
    print(f"All sets completed for all category {i+1}.")

I0000 00:00:1717510222.363366  990888 service.cc:145] XLA service 0x715c640030b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717510222.363394  990888 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3070, Compute Capability 8.6
I0000 00:00:1717510223.494206  990888 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


All sets completed for all category 1.
All sets completed for all category 2.
All sets completed for all category 3.
All sets completed for all category 4.
All sets completed for all category 5.
All sets completed for all category 6.


In [5]:
encoder_models = []
for i in range(6 * 12 * 3):
    encoder_models.append(Model(inputs=input_layer, outputs=encoders[i]))

In [6]:
df = pd.DataFrame(columns=names)
for i in range(6):
    mylist = []
    for j in range(12):
        mylist = mylist + list(encoder_models[12*i+j].predict(vahid_data[i][j])[:,0])
    df[names[i]] = mylist
pd.DataFrame(df).to_csv(f"./processed/vahid_processed.csv")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━

In [7]:
df = pd.DataFrame(columns=names)
for i in range(6):
    mylist = []
    for j in range(12):
        mylist = mylist + list(encoder_models[12*6+12*i+j].predict(amanda_data[i][j])[:,0])
    df[names[i]] = mylist
pd.DataFrame(df).to_csv(f"./processed/amanda_processed.csv")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━

In [8]:
df = pd.DataFrame(columns=names)
for i in range(6):
    mylist = []
    for j in range(12):
        mylist = mylist + list(encoder_models[2*12*6+12*i+j].predict(manuel_data[i][j])[:,0])
    df[names[i]] = mylist
pd.DataFrame(df).to_csv(f"./processed/manuel_processed.csv")

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
8/8 ━━━━━━━━